# Molecular dynamics from NN potentials

## 1. Datset with energy and forces

In [18]:
import numpy as np
from kgcnn.data.datasets.MD17RevisedDataset import MD17RevisedDataset

In [19]:
dataset = MD17RevisedDataset("aspirin")
print("Number of steps", len(dataset))

INFO:kgcnn.data.download:Checking and possibly downloading dataset with name MD17Revised
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.download:Directory for extraction exists. Done.
INFO:kgcnn.data.download:Not extracting tar File. Stopped.


Number of steps 100000


In [21]:
data = dataset[:500]
data.rename_property_on_graphs(old_property_name="energies", new_property_name="energy")
data.rename_property_on_graphs(old_property_name="forces", new_property_name="force")
data.rename_property_on_graphs(old_property_name="nuclear_charges", new_property_name="atomic_number")
data.rename_property_on_graphs(old_property_name="coords", new_property_name="node_coordinates")
data.map_list(method="set_range", node_coordinates="node_coordinates")
data.map_list(method="count_nodes_and_edges", total_edges="total_ranges", count_edges="range_indices", 
    count_nodes="atomic_number", total_nodes="total_nodes")
data[0].keys()

dict_keys(['old_indices', 'old_energies', 'old_forces', 'energy', 'force', 'atomic_number', 'node_coordinates', 'total_nodes', 'range_indices', 'range_attributes', 'total_ranges'])

In [23]:
eng = np.expand_dims(data.get("energy"), axis=-1)*0.043  # kcal/mol to eV
eng.shape, eng[0]

((500, 1), array([-17469.90164208]))

In [25]:
forces = data.tensor({"name": "force", "shape": (None, 3)})*0.043  # kcal/mol to eV
atoms = data.get("atomic_number")
forces[0]

array([[-2.74137341,  2.67700611, -0.75129579],
       [-0.90631249, -0.85888416,  2.00726686],
       [ 3.08056969, -2.19163918, -0.22727128],
       [-0.37739256,  1.81854455, -0.04203819],
       [ 0.06849278,  1.909797  ,  0.88471585],
       [-0.57261954,  0.14867004, -0.18995724],
       [ 0.22410948,  0.77632092, -2.1626258 ],
       [-0.33297714, -1.23917868, -0.10066419],
       [ 0.52634416,  3.8645722 ,  2.08451895],
       [-0.72435461,  1.99949058,  0.7122311 ],
       [ 1.25228829, -1.18763783,  0.57869244],
       [ 0.61277482, -3.80501328, -2.03832495],
       [ 1.67075583, -0.99873548, -0.52365172],
       [-0.30834985, -0.53698349,  0.41058368],
       [ 1.24551168, -1.9163207 ,  1.40335446],
       [ 0.13337422,  0.1260334 , -0.94909162],
       [-1.12054561,  0.36520689, -0.17419492],
       [-0.46182796,  0.5091511 , -0.24017999],
       [-0.29275676, -1.11448041, -0.71522209],
       [-0.47046873,  0.30675033, -0.52878298],
       [-0.50524218, -0.65266984,  0.561

In [26]:
from kgcnn.data.transform.scaler.force import EnergyForceExtensiveLabelScaler
scaler = EnergyForceExtensiveLabelScaler(standardize_coordinates=False, standardize_scale=False)  # For testing no scale.
eng, _ = scaler.fit_transform(X=None, y=eng, force=forces, atomic_number=atoms)
eng[:3]

array([[-0.09150233],
       [ 0.09685228],
       [-0.32295041]])

## 2. `EnergyForceModel` container

In [27]:
from kgcnn.models.force import EnergyForceModel

In [28]:
model_config = {
    "name": "Schnet",
    "coordinate_input": 1,
    "inputs": [
        {"shape": [None], "name": "node_number", "dtype": "int32"},
        {"shape": [None, 3], "name": "node_coordinates", "dtype": "float32"},
        {"shape": [None, 2], "name": "range_indices", "dtype": "int64"},
        {"shape": (), "name": "total_nodes", "dtype": "int64"},
        {"shape": (), "name": "total_ranges", "dtype": "int64"}
    ],
    "nested_model_config": True,
    "output_to_tensor": True,
    "output_squeeze_states": True,
    "model_energy": {
        "class_name": "make_model",
        "module_name": "kgcnn.literature.Schnet",
        "config": {
            "name": "Schnet",
            "inputs": [
                {"shape": [None], "name": "atomic_number", "dtype": "int32"},
                {"shape": [None, 3], "name": "node_coordinates", "dtype": "float32"},
                {"shape": [None, 2], "name": "range_indices", "dtype": "int64"},
                {"shape": (), "name": "total_nodes", "dtype": "int64"},
                {"shape": (), "name": "total_ranges", "dtype": "int64"}
            ],
            "cast_disjoint_kwargs": {"padded_disjoint": False},
            "input_node_embedding": {"input_dim": 95, "output_dim": 64},
            "last_mlp": {"use_bias": [True, True, True], "units": [128, 64, 1],
                         "activation": ['kgcnn>shifted_softplus', 'kgcnn>shifted_softplus', 'linear']},
            "interaction_args": {
                "units": 128, "use_bias": True, "activation": "kgcnn>shifted_softplus",
                "cfconv_pool": "scatter_sum"
            },
            "node_pooling_args": {"pooling_method": "scatter_sum"},
            "depth": 4,
            "gauss_args": {"bins": 20, "distance": 4, "offset": 0.0, "sigma": 0.4}, "verbose": 10,
            "output_embedding": "graph",
            "use_output_mlp": False,
            "output_mlp": None
        }
    },
    "outputs": {"energy": {"name": "energy", "shape": (1,)},
                "force": {"name": "force", "shape": (None, 3)}}

}
model = EnergyForceModel(**model_config)

INFO:kgcnn.models.utils:Updated model kwargs: '{'name': 'Schnet', 'inputs': [{'shape': [None], 'name': 'atomic_number', 'dtype': 'int32'}, {'shape': [None, 3], 'name': 'node_coordinates', 'dtype': 'float32'}, {'shape': [None, 2], 'name': 'range_indices', 'dtype': 'int64'}, {'shape': (), 'name': 'total_nodes', 'dtype': 'int64'}, {'shape': (), 'name': 'total_ranges', 'dtype': 'int64'}], 'cast_disjoint_kwargs': {'padded_disjoint': False}, 'input_node_embedding': {'input_dim': 95, 'output_dim': 64}, 'make_distance': True, 'expand_distance': True, 'interaction_args': {'units': 128, 'use_bias': True, 'activation': 'kgcnn>shifted_softplus', 'cfconv_pool': 'scatter_sum'}, 'node_pooling_args': {'pooling_method': 'scatter_sum'}, 'depth': 4, 'gauss_args': {'bins': 20, 'distance': 4, 'offset': 0.0, 'sigma': 0.4}, 'verbose': 10, 'last_mlp': {'use_bias': [True, True, True], 'units': [128, 64, 1], 'activation': ['kgcnn>shifted_softplus', 'kgcnn>shifted_softplus', 'linear']}, 'output_embedding': 'grap

In [29]:
x_tensor = data.tensor(model_config["model_energy"]["config"]["inputs"])
print([x.shape for x in x_tensor])

[(500, 21), (500, 21, 3), (500, 232, 2), (500,), (500,)]


In [30]:
test_out = model.predict(x_tensor)

16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 140ms/step


In [31]:
[i.shape for i in test_out.values()]

[(500, 1), (500, 21, 3)]

In [32]:
from keras_core.optimizers import Adam

In [42]:
model.compile(
    loss={"energy": "mean_absolute_error", "force": "mean_absolute_error"},
    optimizer=Adam(learning_rate=5e-04),
    metrics=None,
    loss_weights={"energy":1.0, "force":20.0},
)

In [43]:
hist = model.fit(
    x_tensor, {"energy": eng, "force": forces},
    shuffle=True,
    batch_size=64,
    epochs=1500,
    verbose=2,
)

Epoch 1/1500
8/8 - 12s - 2s/step - loss: 19.0471
Epoch 2/1500
8/8 - 3s - 335ms/step - loss: 18.9244
Epoch 3/1500
8/8 - 3s - 336ms/step - loss: 18.9106
Epoch 4/1500
8/8 - 3s - 335ms/step - loss: 18.8388
Epoch 5/1500
8/8 - 3s - 334ms/step - loss: 18.6210
Epoch 6/1500
8/8 - 3s - 319ms/step - loss: 18.6111
Epoch 7/1500
8/8 - 3s - 343ms/step - loss: 18.1707
Epoch 8/1500
8/8 - 3s - 325ms/step - loss: 17.3834
Epoch 9/1500
8/8 - 3s - 333ms/step - loss: 16.9915
Epoch 10/1500
8/8 - 3s - 330ms/step - loss: 17.9063
Epoch 11/1500
8/8 - 3s - 342ms/step - loss: 16.8933
Epoch 12/1500
8/8 - 3s - 351ms/step - loss: 16.1454
Epoch 13/1500
8/8 - 3s - 333ms/step - loss: 16.2273
Epoch 14/1500
8/8 - 3s - 334ms/step - loss: 15.1193
Epoch 15/1500
8/8 - 3s - 321ms/step - loss: 14.8322
Epoch 16/1500
8/8 - 3s - 346ms/step - loss: 13.4976
Epoch 17/1500
8/8 - 3s - 322ms/step - loss: 12.4320
Epoch 18/1500
8/8 - 3s - 341ms/step - loss: 12.0703
Epoch 19/1500
8/8 - 3s - 332ms/step - loss: 11.4993
Epoch 20/1500
8/8 - 3s 

KeyboardInterrupt: 

Epoch 306/1500
8/8 - 1s - loss: 3.1756 - energy_loss: 0.4886 - force_loss: 0.1344 - 543ms/epoch - 68ms/step
Epoch 307/1500
8/8 - 1s - loss: 3.2864 - energy_loss: 0.5515 - force_loss: 0.1367 - 543ms/epoch - 68ms/step
Epoch 308/1500
8/8 - 1s - loss: 3.1964 - energy_loss: 0.5030 - force_loss: 0.1347 - 550ms/epoch - 69ms/step
Epoch 309/1500
8/8 - 1s - loss: 3.2314 - energy_loss: 0.5092 - force_loss: 0.1361 - 536ms/epoch - 67ms/step
Epoch 310/1500
8/8 - 1s - loss: 3.2451 - energy_loss: 0.5154 - force_loss: 0.1365 - 545ms/epoch - 68ms/step
Epoch 311/1500
8/8 - 1s - loss: 3.2679 - energy_loss: 0.4890 - force_loss: 0.1389 - 557ms/epoch - 70ms/step
Epoch 312/1500
8/8 - 1s - loss: 3.3691 - energy_loss: 0.5317 - force_loss: 0.1419 - 567ms/epoch - 71ms/step
Epoch 313/1500
8/8 - 1s - loss: 3.2724 - energy_loss: 0.5227 - force_loss: 0.1375 - 561ms/epoch - 70ms/step
Epoch 314/1500
8/8 - 1s - loss: 3.2920 - energy_loss: 0.5534 - force_loss: 0.1369 - 557ms/epoch - 70ms/step
Epoch 315/1500
8/8 - 1s - lo

Epoch 382/1500
8/8 - 1s - loss: 2.9390 - energy_loss: 0.4811 - force_loss: 0.1229 - 556ms/epoch - 69ms/step
Epoch 383/1500
8/8 - 1s - loss: 3.1760 - energy_loss: 0.4529 - force_loss: 0.1362 - 552ms/epoch - 69ms/step
Epoch 384/1500
8/8 - 1s - loss: 3.4059 - energy_loss: 0.4801 - force_loss: 0.1463 - 560ms/epoch - 70ms/step
Epoch 385/1500
8/8 - 1s - loss: 3.2000 - energy_loss: 0.4863 - force_loss: 0.1357 - 561ms/epoch - 70ms/step
Epoch 386/1500
8/8 - 1s - loss: 3.5248 - energy_loss: 0.9509 - force_loss: 0.1287 - 551ms/epoch - 69ms/step
Epoch 387/1500
8/8 - 1s - loss: 2.7390 - energy_loss: 0.2509 - force_loss: 0.1244 - 545ms/epoch - 68ms/step
Epoch 388/1500
8/8 - 1s - loss: 2.7547 - energy_loss: 0.3570 - force_loss: 0.1199 - 552ms/epoch - 69ms/step
Epoch 389/1500
8/8 - 1s - loss: 2.8498 - energy_loss: 0.4445 - force_loss: 0.1203 - 561ms/epoch - 70ms/step
Epoch 390/1500
8/8 - 1s - loss: 2.9080 - energy_loss: 0.5343 - force_loss: 0.1187 - 558ms/epoch - 70ms/step
Epoch 391/1500
8/8 - 1s - lo

Epoch 458/1500
8/8 - 1s - loss: 2.6934 - energy_loss: 0.4463 - force_loss: 0.1124 - 546ms/epoch - 68ms/step
Epoch 459/1500
8/8 - 1s - loss: 2.6376 - energy_loss: 0.3480 - force_loss: 0.1145 - 551ms/epoch - 69ms/step
Epoch 460/1500
8/8 - 1s - loss: 2.5377 - energy_loss: 0.2556 - force_loss: 0.1141 - 556ms/epoch - 69ms/step
Epoch 461/1500
8/8 - 1s - loss: 2.3334 - energy_loss: 0.1663 - force_loss: 0.1084 - 546ms/epoch - 68ms/step
Epoch 462/1500
8/8 - 1s - loss: 2.3127 - energy_loss: 0.2161 - force_loss: 0.1048 - 545ms/epoch - 68ms/step
Epoch 463/1500
8/8 - 1s - loss: 2.3038 - energy_loss: 0.1915 - force_loss: 0.1056 - 542ms/epoch - 68ms/step
Epoch 464/1500
8/8 - 1s - loss: 2.5906 - energy_loss: 0.4175 - force_loss: 0.1087 - 552ms/epoch - 69ms/step
Epoch 465/1500
8/8 - 1s - loss: 2.4632 - energy_loss: 0.2212 - force_loss: 0.1121 - 544ms/epoch - 68ms/step
Epoch 466/1500
8/8 - 1s - loss: 2.3895 - energy_loss: 0.1886 - force_loss: 0.1100 - 547ms/epoch - 68ms/step
Epoch 467/1500
8/8 - 1s - lo

...

Epoch 1142/1500
8/8 - 1s - loss: 1.7628 - energy_loss: 0.3109 - force_loss: 0.0726 - 568ms/epoch - 71ms/step
Epoch 1143/1500
8/8 - 1s - loss: 1.8375 - energy_loss: 0.3384 - force_loss: 0.0750 - 573ms/epoch - 72ms/step
Epoch 1144/1500
8/8 - 1s - loss: 1.8913 - energy_loss: 0.3271 - force_loss: 0.0782 - 573ms/epoch - 72ms/step
Epoch 1145/1500
8/8 - 1s - loss: 1.9532 - energy_loss: 0.3296 - force_loss: 0.0812 - 571ms/epoch - 71ms/step
Epoch 1146/1500
8/8 - 1s - loss: 1.7836 - energy_loss: 0.1518 - force_loss: 0.0816 - 572ms/epoch - 72ms/step
Epoch 1147/1500
8/8 - 1s - loss: 1.6513 - energy_loss: 0.1623 - force_loss: 0.0744 - 569ms/epoch - 71ms/step
Epoch 1148/1500
8/8 - 1s - loss: 1.5832 - energy_loss: 0.1443 - force_loss: 0.0719 - 574ms/epoch - 72ms/step
Epoch 1149/1500
8/8 - 1s - loss: 1.5962 - energy_loss: 0.1546 - force_loss: 0.0721 - 569ms/epoch - 71ms/step
Epoch 1150/1500
8/8 - 1s - loss: 1.5537 - energy_loss: 0.1331 - force_loss: 0.0710 - 566ms/epoch - 71ms/step
Epoch 1151/1500
8/8

Epoch 1218/1500
8/8 - 1s - loss: 1.8765 - energy_loss: 0.3325 - force_loss: 0.0772 - 569ms/epoch - 71ms/step
Epoch 1219/1500
8/8 - 1s - loss: 1.8138 - energy_loss: 0.3137 - force_loss: 0.0750 - 608ms/epoch - 76ms/step
Epoch 1220/1500
8/8 - 1s - loss: 1.8041 - energy_loss: 0.3196 - force_loss: 0.0742 - 575ms/epoch - 72ms/step
Epoch 1221/1500
8/8 - 1s - loss: 1.8811 - energy_loss: 0.3093 - force_loss: 0.0786 - 567ms/epoch - 71ms/step
Epoch 1222/1500
8/8 - 1s - loss: 1.8883 - energy_loss: 0.3338 - force_loss: 0.0777 - 564ms/epoch - 71ms/step
Epoch 1223/1500
8/8 - 1s - loss: 1.8464 - energy_loss: 0.3251 - force_loss: 0.0761 - 569ms/epoch - 71ms/step
Epoch 1224/1500
8/8 - 1s - loss: 1.8031 - energy_loss: 0.3220 - force_loss: 0.0741 - 568ms/epoch - 71ms/step
Epoch 1225/1500
8/8 - 1s - loss: 1.7746 - energy_loss: 0.3093 - force_loss: 0.0733 - 573ms/epoch - 72ms/step
Epoch 1226/1500
8/8 - 1s - loss: 1.7120 - energy_loss: 0.3208 - force_loss: 0.0696 - 577ms/epoch - 72ms/step
Epoch 1227/1500
8/8

Epoch 1294/1500
8/8 - 1s - loss: 1.8698 - energy_loss: 0.3706 - force_loss: 0.0750 - 610ms/epoch - 76ms/step
Epoch 1295/1500
8/8 - 1s - loss: 1.8268 - energy_loss: 0.2774 - force_loss: 0.0775 - 645ms/epoch - 81ms/step
Epoch 1296/1500
8/8 - 1s - loss: 1.7237 - energy_loss: 0.3004 - force_loss: 0.0712 - 611ms/epoch - 76ms/step
Epoch 1297/1500
8/8 - 1s - loss: 1.7796 - energy_loss: 0.3744 - force_loss: 0.0703 - 603ms/epoch - 75ms/step
Epoch 1298/1500
8/8 - 1s - loss: 1.6922 - energy_loss: 0.2727 - force_loss: 0.0710 - 598ms/epoch - 75ms/step
Epoch 1299/1500
8/8 - 1s - loss: 1.7245 - energy_loss: 0.2860 - force_loss: 0.0719 - 614ms/epoch - 77ms/step
Epoch 1300/1500
8/8 - 1s - loss: 1.8538 - energy_loss: 0.2733 - force_loss: 0.0790 - 618ms/epoch - 77ms/step
Epoch 1301/1500
8/8 - 1s - loss: 1.9914 - energy_loss: 0.3206 - force_loss: 0.0835 - 652ms/epoch - 81ms/step
Epoch 1302/1500
8/8 - 1s - loss: 2.0053 - energy_loss: 0.3555 - force_loss: 0.0825 - 633ms/epoch - 79ms/step
Epoch 1303/1500
8/8

Epoch 1370/1500
8/8 - 1s - loss: 1.7885 - energy_loss: 0.3029 - force_loss: 0.0743 - 584ms/epoch - 73ms/step
Epoch 1371/1500
8/8 - 1s - loss: 1.8967 - energy_loss: 0.3072 - force_loss: 0.0795 - 579ms/epoch - 72ms/step
Epoch 1372/1500
8/8 - 1s - loss: 1.7590 - energy_loss: 0.3019 - force_loss: 0.0729 - 668ms/epoch - 83ms/step
Epoch 1373/1500
8/8 - 1s - loss: 1.7160 - energy_loss: 0.3050 - force_loss: 0.0706 - 657ms/epoch - 82ms/step
Epoch 1374/1500
8/8 - 1s - loss: 1.7361 - energy_loss: 0.2934 - force_loss: 0.0721 - 631ms/epoch - 79ms/step
Epoch 1375/1500
8/8 - 1s - loss: 1.7108 - energy_loss: 0.3144 - force_loss: 0.0698 - 656ms/epoch - 82ms/step
Epoch 1376/1500
8/8 - 1s - loss: 1.7204 - energy_loss: 0.2977 - force_loss: 0.0711 - 671ms/epoch - 84ms/step
Epoch 1377/1500
8/8 - 1s - loss: 1.7563 - energy_loss: 0.2909 - force_loss: 0.0733 - 668ms/epoch - 84ms/step
Epoch 1378/1500
8/8 - 1s - loss: 1.6346 - energy_loss: 0.2272 - force_loss: 0.0704 - 677ms/epoch - 85ms/step
Epoch 1379/1500
8/8

Epoch 1446/1500
8/8 - 1s - loss: 1.7595 - energy_loss: 0.3143 - force_loss: 0.0723 - 668ms/epoch - 83ms/step
Epoch 1447/1500
8/8 - 1s - loss: 1.7839 - energy_loss: 0.2968 - force_loss: 0.0744 - 637ms/epoch - 80ms/step
Epoch 1448/1500
8/8 - 1s - loss: 1.8501 - energy_loss: 0.3171 - force_loss: 0.0766 - 607ms/epoch - 76ms/step
Epoch 1449/1500
8/8 - 1s - loss: 1.8423 - energy_loss: 0.3111 - force_loss: 0.0766 - 599ms/epoch - 75ms/step
Epoch 1450/1500
8/8 - 1s - loss: 1.7719 - energy_loss: 0.3149 - force_loss: 0.0729 - 590ms/epoch - 74ms/step
Epoch 1451/1500
8/8 - 1s - loss: 1.6898 - energy_loss: 0.3050 - force_loss: 0.0692 - 605ms/epoch - 76ms/step
Epoch 1452/1500
8/8 - 1s - loss: 1.7285 - energy_loss: 0.3019 - force_loss: 0.0713 - 587ms/epoch - 73ms/step
Epoch 1453/1500
8/8 - 1s - loss: 1.7353 - energy_loss: 0.2921 - force_loss: 0.0722 - 631ms/epoch - 79ms/step
Epoch 1454/1500
8/8 - 1s - loss: 1.7460 - energy_loss: 0.2918 - force_loss: 0.0727 - 610ms/epoch - 76ms/step
Epoch 1455/1500
8/8

In [ ]:
from kgcnn.utils.plots import plot_train_test_loss

In [ ]:
plot_train_test_loss([hist]);

## 3. `MolDynamicsModelPredictor` wrapping pre- and postprocessing

In [ ]:
from kgcnn.molecule.dynamics.base import MolDynamicsModelPredictor
from kgcnn.graph.postprocessor import ExtensiveEnergyForceScalerPostprocessor
from kgcnn.graph.preprocessor import SetRange

In [37]:
dyn_model = MolDynamicsModelPredictor(
    model=model, 
    model_inputs=model_config["model_energy"]["config"]["inputs"], 
    model_outputs={"energy":"energy", "forces": "force"},
    graph_preprocessors=[SetRange(node_coordinates="node_coordinates", overwrite=False)],  # To use 
    graph_postprocessors=[
        ExtensiveEnergyForceScalerPostprocessor(
            scaler, energy="energy", force="forces", atomic_number="nuclear_charges")],
    store_last_input=True,
    update_from_last_input=["range_indices"],  # and "range_images" if periodic.
    update_from_last_input_skip=3    
)

In [38]:
dyn_model(dataset[0:510])[0]

TypeError: 'NoneType' object is not subscriptable

## 4. Use ASE compatible `KgcnnSingleCalculator`

In [39]:
from ase import Atoms
from kgcnn.molecule.dynamics.ase_calc import KgcnnSingleCalculator, AtomsToGraphConverter

In [ ]:
atoms = Atoms(dataset[0]["nuclear_charges"], positions=dataset[0]["coords"])
atoms

In [ ]:
conv=AtomsToGraphConverter({"nuclear_charges": "get_atomic_numbers", "coords": "get_positions"})
conv(atoms)

In [ ]:
calc = KgcnnSingleCalculator(model_predictor=dyn_model, atoms_converter=conv)
calc

In [ ]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase import units

In [ ]:
atoms.calc = calc
calc.calculate(atoms)
print(calc.results)

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(atoms, temperature_K=300)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, 1 * units.fs)  # 5 fs time step.

In [ ]:
def printenergy(a):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))


# Now run the dynamics
printenergy(atoms)
for i in range(20):
    dyn.run(10)
    printenergy(atoms)

In [ ]:
calc.model_predictor._last_input;

In [ ]:
dyn_model._test_timing(conv(atoms))